In [6]:
!pip install pandas

In [86]:
!pip install pickle-mixin

  Preparing metadata (setup.py) ... done
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=5991 sha256=04d8a37b1f706bd2c5bf8884d6b9c4d483e848d00c3c47b6621a1528e34113a4
  Stored in directory: /Users/vithalbeeranagaddi/Library/Caches/pip/wheels/69/e2/5c/da8f96a08c63469bc8b10e206cd4c78e8886d8acb8699f84c2
Successfully built pickle-mixin


In [114]:
import pandas as pd
from string import ascii_uppercase as alphabet
import pickle

In [94]:
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

In [96]:
all_tables[9]
all_tables[10]
all_tables[11]
all_tables[12]
all_tables[16]

,Pos,Team
0,H1,Portugal
1,H2,Ghana
2,H3,Uruguay
3,H4,South Korea


In [98]:
dict_table = {}
for letter, i in zip(alphabet,range(9,17)):
    df = all_tables[i]
    dict_table[f'Group {letter}'] = df

In [100]:
dict_table['Group H']

,Pos,Team
0,H1,Portugal
1,H2,Ghana
2,H3,Uruguay
3,H4,South Korea


In [120]:
with open('dict_table', 'wb') as output:
    pickle.dump(dict_table, output)

In [122]:
!pip install bs4

In [351]:
from bs4 import BeautifulSoup
import requests

years = [1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,1982,1986,1990,1994,1998,2002,2006,2010,2014,2018,2022]

def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content,'lxml')
    matches = soup.find_all('div',class_='footballbox')
    
    home = []
    score = []
    away = []
    
    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())
    
    dict_football = {'Home':home, 'Score':score, 'Away':away}
    df_football = pd.DataFrame(dict_football)
    df_football['Year'] = year
    return df_football

#Historical data
fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('fifaworldcup_historical_data.csv', index=False)


In [353]:
# Data Cleaning

df_historical = pd.read_csv('fifaworldcup_historical_data.csv')

In [355]:
df_fixture['Home'] = df_fixture['Home'].str.strip()


In [357]:
# to check for null values df_historical[df_historical['Score'].isnull()]
# in case of any null values/missing values drop it by using df_historical.dropna(inplace=True)

df_historical[df_historical['Score'].isnull()]

,Home,Score,Away,Year


In [359]:
# To check for duplicates, if exists then drop it by using df_historical.drop_duplicates(inplace=True)
duplicated_rows = df_historical.duplicated()
duplicated_rows

0      False
1      False
2      False
3      False
4      False
       ...  
924    False
925    False
926    False
927    False
928    False
Length: 929, dtype: bool

In [361]:
df_historical[df_historical['Home'].str.contains('Sweden') & df_historical['Away'].str.contains('Austria')]

,Home,Score,Away,Year
37,Sweden,w/o,Austria,1938


In [363]:
#Deleting the above data as it contains w/o in score column

delete_index = df_historical[df_historical['Home'].str.contains('Sweden') & df_historical['Away'].str.contains('Austria')].index

df_historical.drop(index = delete_index, inplace = True)

In [365]:
df_historical[df_historical['Home'].str.contains('Sweden') & df_historical['Away'].str.contains('Austria')]

,Home,Score,Away,Year


In [367]:
#column score not only with digit and '-' 
df_historical['Score']=df_historical['Score'].str.replace(r'[^\d-]','',regex=True)



In [369]:
df_historical['Home'] = df_historical['Home'].str.strip()
df_historical['Away'] = df_historical['Away'].str.strip()
df_historical['Score'] = df_historical['Score'].str.strip()

In [371]:
#Splitting the Scores as Home Goals and Away Goals
df_historical[['0','HomeGoals','AwayGoals','3','None']] = df_historical['Score'].str.split('', expand=True)

In [373]:
df_historical

,Home,Score,Away,Year,0,HomeGoals,AwayGoals,3,None
0,France,41,Mexico,1930,,4,1,,None
1,Argentina,10,France,1930,,1,0,,None
2,Chile,30,Mexico,1930,,3,0,,None
3,Chile,10,France,1930,,1,0,,None
4,Argentina,63,Mexico,1930,,6,3,,None
...,...,...,...,...,...,...,...,...,...
924,England,12,France,2022,,1,2,,None
925,Argentina,30,Croatia,2022,,3,0,,None
926,France,20,Morocco,2022,,2,0,,None
927,Croatia,21,Morocco,2022,,2,1,,None


In [375]:
#Deleting unnecessary columns
df_historical = df_historical.drop(columns=['0','3','None'])

In [377]:
#Deleting Score column
df_historical.drop('Score',axis=1, inplace=True)

In [379]:
df_historical

,Home,Away,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
924,England,France,2022,1,2
925,Argentina,Croatia,2022,3,0
926,France,Morocco,2022,2,0
927,Croatia,Morocco,2022,2,1


In [381]:
#Renaming Home and Away columns
df_historical.rename(columns={'Home':'HomeTeam','Away':'AwayTeam'},inplace=True)

In [383]:
df_historical

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
924,England,France,2022,1,2
925,Argentina,Croatia,2022,3,0
926,France,Morocco,2022,2,0
927,Croatia,Morocco,2022,2,1


In [385]:
df_historical.dtypes

HomeTeam     object
AwayTeam     object
Year          int64
HomeGoals    object
AwayGoals    object
dtype: object

In [387]:
#Changing the datatypes

df_historical = df_historical.astype({'HomeGoals': 'int', 'AwayGoals': 'int', 'Year': 'int'})

In [389]:
df_historical.dtypes

HomeTeam     object
AwayTeam     object
Year          int64
HomeGoals     int64
AwayGoals     int64
dtype: object

In [391]:
#Total Goals

df_historical['TotalGoals'] = df_historical['HomeGoals'] + df_historical['AwayGoals']

In [393]:
df_historical

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
924,England,France,2022,1,2,3
925,Argentina,Croatia,2022,3,0,3
926,France,Morocco,2022,2,0,2
927,Croatia,Morocco,2022,2,1,3


In [395]:
df_historical.to_csv('clean_fifaworldcup_historical_data.csv', index=False)


In [399]:
#Extra Verification

years = [1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,1982,1986,1990,1994,1998,2002,2006,2010,2014,2018,2022]

for year in years:
    print(year, len(df_historical[df_historical['Year']==year]))

1930 18
1934 17
1938 18
1950 22
1954 26
1958 35
1962 32
1966 32
1970 32
1974 38
1978 38
1982 52
1986 52
1990 16
1994 52
1998 64
2002 64
2006 64
2010 64
2014 64
2018 64
2022 64
